In [ ]:
using Pkg; Pkg.activate()

# Convergence of CMB power spectrum

In [ ]:
using SymBoltz, Plots
M = ΛCDM(ν = nothing, h = nothing)
pars = SymBoltz.parameters_Planck18(M)
prob = CosmologyProblem(M, pars)

In [ ]:
ls = 25:25:1000
lmin = ls[begin]
lmax = ls[end]

p = plot()
for Nlos in [512, 768, 1024, 2048, 0] # number of time points for LOS integration (0 for "adaptive")
for kτ0min in [0.1] .* lmin # minimum wavenumber*timetoday to integrate in k
for kτ0max in [3] .* lmax # maximum wavenumber*timetoday to integrate in k
for Δkτ0 in [2π/2] # density of wavenumber*timetoday to integrate in k (and to interpolate ODE solutions)
for Δkτ0_S in [8.0] # density of wavenumbers to solve ODEs with
for bgreltol in [1e-8] # tolerance for background ODE solver
for ptreltol in [1e-8] # tolerance for perturbation ODE solver
for bgalg in [SymBoltz.Rodas4P()] # background ODE solver
for ptalg in [SymBoltz.KenCarp4()] # perturbation ODE solver
for integrator in [SymBoltz.TrapezoidalRule()] # integration method for LOS and k integration
for ktransform in [identity] # linearly interpolate solution between ktransform(k1) and ktransform(k2)
    Dls = spectrum_cmb(:TT, prob, ls; normalization = :Dl, Δkτ0, Δkτ0_S, kτ0min, kτ0max, Nlos, bgopts = (alg = bgalg, reltol = bgreltol), ptopts = (alg = ptalg, reltol = ptreltol,), integrator, ktransform, verbose = true)
    Plots.plot!(p, ls, Dls, linestyle = :solid, alpha = 0.8, xlabel = "l", ylabel = "Dₗ")
end end end end end end end end end end end
plot(p; fmt = :png)

1. Change $S_0$ and $S_1$ in code.
2. Calculate CMB power spectrum for e.g. `Nlos = 0` and `Nlos = 1024`.
3. Check stability.

Results (for $l \gtrsim 25$):

- **(I)SW:**
  - $v(+\Psi) + e^{-\kappa}(\Psi+\Phi)^\prime$ (original) is stable around `Nlos >= 512`.
  - $v(+\Phi) + 2 e^{-\kappa} \Phi^\prime + k \cdot e^{-\kappa}(\Psi-\Phi)$ is stable around `Nlos >= 512`.
  - $v(-\Phi) + k \cdot e^{-\kappa}(\Psi+\Phi)$ is very unstable.
- **Doppler:**
  - 0 IBP is stable around `Nlos = 1024`.
  - 1 IBP is slightly unstable `Nlos = 1024`.
- **Polarization:** 0 IBP and 1 IBP are equally stable when changing `Nlos`.

So might as well use 0 integrations by parts in all terms...

In [ ]:
# TODO: repeat for ∂Dₗ/∂θᵢ
# TODO: repeat for TE and EE